# Configuração de Acesso ao MinIO

Este notebook configura o acesso ao bucket `govbr` no MinIO.

## 1. Instalação da biblioteca minio

Primeiro, vamos instalar a biblioteca cliente do MinIO para Python.

In [ ]:
!pip install minio

## 2. Importar bibliotecas e configurar credenciais

In [ ]:
from minio import Minio
from minio.error import S3Error
import os

# Configurações do MinIO
MINIO_SERVER_URL = "ch8ai-minio.l6zv5a.easypanel.host"
MINIO_ROOT_USER = "admin"
MINIO_ROOT_PASSWORD = "1q2w3e4r"
BUCKET_NAME = "govbr"

print("Credenciais configuradas com sucesso!")

## 3. Conectar ao MinIO

Criar cliente MinIO e testar a conexão.

In [ ]:
# Criar cliente MinIO
client = Minio(
    MINIO_SERVER_URL,
    access_key=MINIO_ROOT_USER,
    secret_key=MINIO_ROOT_PASSWORD,
    secure=True  # Usando HTTPS
)

print(f"Cliente MinIO criado com sucesso!")
print(f"Servidor: {MINIO_SERVER_URL}")

## 4. Verificar se o bucket existe

In [ ]:
try:
    # Verificar se o bucket existe
    if client.bucket_exists(BUCKET_NAME):
        print(f"✓ Bucket '{BUCKET_NAME}' existe e está acessível!")
    else:
        print(f"✗ Bucket '{BUCKET_NAME}' não existe.")
        # Criar bucket se não existir
        client.make_bucket(BUCKET_NAME)
        print(f"✓ Bucket '{BUCKET_NAME}' criado com sucesso!")
except S3Error as e:
    print(f"Erro ao acessar o bucket: {e}")

## 5. Listar todos os buckets disponíveis

In [ ]:
try:
    buckets = client.list_buckets()
    print("Buckets disponíveis:")
    for bucket in buckets:
        print(f"  - {bucket.name} (criado em: {bucket.creation_date})")
except S3Error as e:
    print(f"Erro ao listar buckets: {e}")

## 6. Listar objetos no bucket govbr

In [ ]:
try:
    objects = client.list_objects(BUCKET_NAME, recursive=True)
    print(f"Objetos no bucket '{BUCKET_NAME}':")
    count = 0
    for obj in objects:
        print(f"  - {obj.object_name} (tamanho: {obj.size} bytes, última modificação: {obj.last_modified})")
        count += 1
    
    if count == 0:
        print("  (vazio)")
    else:
        print(f"\nTotal: {count} objetos")
except S3Error as e:
    print(f"Erro ao listar objetos: {e}")

## 7. Exemplo: Upload de arquivo

In [ ]:
# Criar um arquivo de teste
test_file = "test_file.txt"
with open(test_file, "w") as f:
    f.write("Este é um arquivo de teste para o bucket govbr no MinIO.")

try:
    # Upload do arquivo
    client.fput_object(
        BUCKET_NAME,
        "test/test_file.txt",  # Nome do objeto no bucket
        test_file  # Arquivo local
    )
    print(f"✓ Arquivo '{test_file}' enviado com sucesso para '{BUCKET_NAME}/test/test_file.txt'")
    
    # Remover arquivo local
    os.remove(test_file)
except S3Error as e:
    print(f"Erro ao fazer upload: {e}")

## 8. Exemplo: Download de arquivo

In [ ]:
try:
    # Download do arquivo
    downloaded_file = "downloaded_test_file.txt"
    client.fget_object(
        BUCKET_NAME,
        "test/test_file.txt",  # Nome do objeto no bucket
        downloaded_file  # Arquivo local para salvar
    )
    print(f"✓ Arquivo baixado com sucesso para '{downloaded_file}'")
    
    # Ler conteúdo
    with open(downloaded_file, "r") as f:
        content = f.read()
        print(f"\nConteúdo: {content}")
    
    # Remover arquivo local
    os.remove(downloaded_file)
except S3Error as e:
    print(f"Erro ao fazer download: {e}")

## 9. Exemplo: Remover objeto

In [ ]:
try:
    # Remover objeto
    client.remove_object(BUCKET_NAME, "test/test_file.txt")
    print(f"✓ Objeto 'test/test_file.txt' removido com sucesso")
except S3Error as e:
    print(f"Erro ao remover objeto: {e}")

## 10. Funções auxiliares úteis

In [ ]:
def get_object_info(bucket_name, object_name):
    """Obtém informações sobre um objeto específico"""
    try:
        stat = client.stat_object(bucket_name, object_name)
        print(f"Informações do objeto '{object_name}':")
        print(f"  - Tamanho: {stat.size} bytes")
        print(f"  - ETag: {stat.etag}")
        print(f"  - Content-Type: {stat.content_type}")
        print(f"  - Última modificação: {stat.last_modified}")
        return stat
    except S3Error as e:
        print(f"Erro ao obter informações: {e}")
        return None

def get_presigned_url(bucket_name, object_name, expires_seconds=3600):
    """Gera URL pré-assinada para acesso temporário ao objeto"""
    try:
        url = client.presigned_get_object(bucket_name, object_name, expires=expires_seconds)
        print(f"URL pré-assinada (válida por {expires_seconds} segundos):")
        print(url)
        return url
    except S3Error as e:
        print(f"Erro ao gerar URL: {e}")
        return None

print("Funções auxiliares carregadas!")